In [1]:
%pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 8.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
os.chdir("/home/ec2-user/SageMaker/")

In [ ]:
os.listdir()

In [23]:
# %%sh
# cd ./custom_inference/
# tar zcvf model.tar.gz *

adapter_config.json
adapter_model.safetensors
code/
code/requirements.txt
code/inference.py
code/.ipynb_checkpoints/
code/.ipynb_checkpoints/inference-checkpoint.py
code/.ipynb_checkpoints/requirements-checkpoint.txt


In [ ]:
# !aws s3 cp ./custom_inference/model.tar.gz s3://sagemaker-eu-west-1-211125449279/LLM-Textmarker-mistralai-Mistral-7B-Instruct-v0-2-16-43-42/checkpoints/custom_inference/

In [9]:
s3_location = "s3://sagemaker-eu-west-1-211125449279/LLM-Textmarker-mistralai-Mistral-7B-Instruct-v0-2-16-43-42/checkpoints/custom_inference/model.tar.gz"

In [10]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::211125449279:role/service-role/AmazonSageMaker-ExecutionRole-20240307T175168
sagemaker bucket: sagemaker-eu-west-1-211125449279
sagemaker session region: eu-west-1


In [11]:
from sagemaker.huggingface.model import HuggingFaceModel
from dotenv import load_dotenv

load_dotenv()
hf_token = os.environ.get("API_TOKEN", None)
env_dict = {
    "HF_TOKEN": hf_token
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=s3_location,       # path to your model and script
   role=role,                    # iam role with permissions to create an Endpoint
   transformers_version="4.37",  # transformers version used
   pytorch_version="2.1",        # pytorch version used
   env=env_dict, # env variables
   py_version='py310',            # python version used
)

# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge"
    )

---------!

In [12]:
from datasets import load_dataset

# dataset used
data_path = 'MichaelAI23/English_CVs'

data = load_dataset(data_path)

train_val = data["train"].train_test_split(
    test_size=100, shuffle=True, seed=42
)

train_data = train_val["train"]
val_data = train_val["test"]

In [13]:
val_data[0]["overall"]

'As a Business Analyst at TechSolutions Inc., I was responsible for analyzing market trends and providing strategic recommendations to drive business growth. I started my role in January 2014 after graduating in 2012 and continued until December 2018, where I successfully implemented data-driven solutions that improved operational efficiency.\n- Published research paper on "The Impact of Data Analytics in Business Decision Making" in Journal of Business Intelligence, 2012\n- Co-authored a study on "Predictive Modeling for Customer Churn Analysis" in International Conference on Business Analytics, 2013\n- Presented findings on "Optimizing Supply Chain Management through Big Data Analysis" at the Annual Meeting of Operations Research Society, 2014\n- Contributed to a book chapter titled "Emerging Trends in Business Forecasting Techniques" published by Springer, 2015\n- Received Best Paper Award for research on "Machine Learning Applications in Marketing Strategy Development" at the Acade

In [47]:
import time
start_time = time.time()
data = {
    "inputs": val_data[32]["overall"],
    "parameters": {"max_new_tokens": 128, "temperature": 0}
}

res = predictor.predict(data=data)
print("--- %s seconds ---" % (time.time() - start_time))

print(res)


--- 3.30521821975708 seconds ---
[{'generated_text': " {'pe': {'s': '- Mail', 'e': '/alessandrojamirez'}, 'ed': {'s': 'A bachelor', 'e': ' completed.'}, 'wo': {'s': 'Data Engineer, 2014', 'e': ' datasets.'}, 'sk': {'s': 'Experienced', 'e': ' capabilities.'}}"}]


In [37]:
res[0]

{'generated_text': " {'pe': {'s': 'Avery', 'e': ' Laurier.'}, 'ed': {'s': 'Juris', 'e': ', 2018'}, 'wo': {'s': 'Legal', 'e': ' initiatives.'}, 'sk': {'s': 'Experienced', 'e': ' for clients.'}}"}

In [33]:
# delete endpoint
predictor.delete_endpoint()